# Test Functions for Kriging

In [ ]:
from spotPython.build.kriging import Kriging
import numpy as np
import matplotlib.pyplot as plt
from numpy import linspace, arange
rng = np.random.RandomState(1)
X = linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
y = np.squeeze(X * np.sin(X))
training_indices = rng.choice(arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]
S = Kriging(name='kriging', seed=124)
S.fit(X_train, y_train)
mean_prediction, std_prediction, s_ei = S.predict(X, return_val="all")
plt.plot(X, y, label=r"$f(x)$", linestyle="dotted")
plt.scatter(X_train, y_train, label="Observations")
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
    )
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on noise-free dataset")
plt.show()

In [ ]:
from spotPython.build.kriging import Kriging
import numpy as np
import matplotlib.pyplot as plt
from numpy import linspace, arange
rng = np.random.RandomState(1)
X = linspace(start=0, stop=10, num=1_0).reshape(-1, 1)
y = np.squeeze(X * np.sin(X))
training_indices = rng.choice(arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]
S = Kriging(name='kriging', seed=124)
S.fit(X_train, y_train)
mean_prediction, std_prediction, s_ei = S.predict(X, return_val="all")
# Kriging is a interpolator, so the mean prediction should be equal to the training points:
# check if the difference between the mean prediction and the true value in the training points is smaller than 1e-6
assert np.allclose(mean_prediction[training_indices], y[training_indices], atol=1e-6)


## Expected Improvement

In [ ]:
from spotPython.build.kriging import Kriging
from math import erf
import numpy as np
S = Kriging(name='kriging', seed=124)
S.mean_cod_y = [0.0, 0.0, 0.0, 0.0, 0.0]
# asset that the S.exp_imp(1.0, 0.0) is equal to 0.0
assert 0.0 == S.exp_imp(1.0, 0.0)
# assert that the S.exp_imp(0.0, 1.0) is equal to 1/sqrt(2 pi)
# assert S.exp_imp(0.0, 1.0) == 1/np.sqrt(2*np.pi)
# play safe and use np.allclose
assert np.allclose(S.exp_imp(0.0, 1.0), 1/np.sqrt(2*np.pi), atol=1e-6)
assert np.allclose(S.exp_imp(1.0, 1.0), -(0.5 + 0.5*erf(-1/np.sqrt(2))) + 1/np.sqrt(2*np.pi)*np.exp(-1/2), atol=1e-6)

# set_de_bounds

In [ ]:
from spotPython.build.kriging import Kriging
S = Kriging(name='kriging', seed=124)
S.set_de_bounds()
assert S.de_bounds == [[-3, 2]]
from spotPython.build.kriging import Kriging
n = 10
S = Kriging(name='kriging', seed=124, n_theta=n)
S.set_de_bounds()
assert len(S.de_bounds) == n
n=2
p=4
S = Kriging(name='kriging', seed=124, n_theta=n, n_p=p, optim_p=True)
S.set_de_bounds()
assert len(S.de_bounds) == n+p
S = Kriging(name='kriging', seed=124, n_theta=n, n_p=p, optim_p=False)
S.set_de_bounds()
assert len(S.de_bounds) == n

## extract_from_bounds

In [21]:
import numpy as np
from spotPython.build.kriging import Kriging
n=2
p=2
S = Kriging(name='kriging', seed=124, n_theta=n, n_p=p, optim_p=True, noise=False)
S.extract_from_bounds(np.array([1, 2, 3]))
assert len(S.theta) == n





## optimize_model

In [19]:
from spotPython.build.kriging import Kriging
import numpy as np
nat_X = np.array([[1, 2], [3, 4]])
nat_y = np.array([1, 2])
n=2
p=2
S=Kriging(name='kriging', seed=124, n_theta=n, n_p=p, optim_p=True, noise=True)
S.initialize_variables(nat_X, nat_y)
S.set_variable_types()
S.nat_to_cod_init()
S.set_theta_values()
S.initialize_matrices()
S.set_de_bounds()
new_theta_p_Lambda = S.optimize_model()
assert len(new_theta_p_Lambda) == n+p+1
# no noise, so Lambda is not considered
S=Kriging(name='kriging', seed=124, n_theta=n, n_p=p, optim_p=True, noise=False)
S.initialize_variables(nat_X, nat_y)
S.set_variable_types()
S.nat_to_cod_init()
S.set_theta_values()
S.initialize_matrices()
S.set_de_bounds()
new_theta_p_Lambda = S.optimize_model()
assert len(new_theta_p_Lambda) == n+p